In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, SubsetRandomSampler

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, KFold

from torch_lr_finder import LRFinder

from data import preprocessing, preprocessing, preprocessing2
from modelling import LSTMModel_base, train_epoch, valid_epoch, L1Loss_masked
from func import get_timestamp

import timeit, copy

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
data_kwargs = {'u_in_cumsum': True,
          'u_in_lag12': True,
          'u_in_lag34': False,
          'u_in_lag_back12': True,
          'u_in_lag_back34': False,
          'u_in_diff12': True,
          'u_in_diff34': False,
          'u_in_diff_back12': True,
          'u_in_diff_back34': False,
          'u_in_last': False,
          'u_in_max': False,
          'scaler': RobustScaler()}
train, test, features = preprocessing(train, test, **data_kwargs)

In [ ]:
train

In [ ]:
target = train['pressure']
train.drop(columns = ['id', 'breath_id', 'pressure'], inplace = True)
test.drop(columns = ['id', 'breath_id'], inplace = True)
input_size = train.shape[1]
features, train.columns, input_size

In [ ]:
# Creating dataset
train = torch.tensor(train.to_numpy()).reshape(-1,80, input_size).float()
test = torch.tensor(test.to_numpy()).reshape(-1,80, input_size).float()
target = torch.tensor(target.to_numpy()).reshape(-1,80,1).float()

train_dataset = TensorDataset(train, target)

In [ ]:
class Config:
    lr = 1e-03
    
    batch_size = 256
    num_workers = 4
    device = "cuda"
    
    num_epochs = 500
    k = 15
    
    lr_finder = False
    training = True
    
cfg = Config

## LR finder

In [ ]:
if cfg.lr_finder:
    model = LSTMModel_base(input_size)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters())
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, num_workers=cfg.num_workers)
    lr_finder = LRFinder(model, optimizer, criterion, device=cfg.device)
    lr_finder.range_test(train_loader, start_lr = 0.00001, end_lr=1, num_iter=500)
    plt.rcParams["figure.figsize"] = (10,4)
    lr_finder.plot() # to inspect the loss-learning rate graph
    lr_finder.reset() 

## Train 1 fold

In [ ]:
kf=KFold(n_splits=cfg.k,shuffle=True,random_state=123)
kf = list(kf.split(train))
for i in range(cfg.k):
    print(kf[i])

In [ ]:
folds = [2, 3]

opt_criterion = L1Loss_masked()
val_criterion = L1Loss_masked()

device = cfg.device

for fold in folds:
    if cfg.training:    
        train_idx,val_idx = kf[fold]
        print(f"Fold {fold}", "\n")

        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, sampler=train_sampler, num_workers = cfg.num_workers)
        val_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, sampler=val_sampler, num_workers = cfg.num_workers)

        model = LSTMModel_base(input_size)
        model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=0.01)
        batch_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 50, T_mult = 1, eta_min = 1e-05)
        #batch_scheduler = None
        #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5)
        scheduler = None
        best_val_loss = 1000
        best_weights = model.state_dict()
        restart = 15

        beg_time = timeit.default_timer()

        for epoch in range(cfg.num_epochs):
            start_time = timeit.default_timer()
            train_loss = train_epoch(model,device,train_loader,opt_criterion,optimizer, epoch, batch_scheduler)
            val_loss = valid_epoch(model,device,val_loader,val_criterion)
            end_time = timeit.default_timer()

            total = end_time - start_time

            train_loss = np.mean(np.array(train_loss))
            val_loss = np.mean(np.array(val_loss))

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_weights = copy.copy(model.state_dict())

            print(f"Epoch: {epoch} | T loss: {train_loss:.4f} V loss: {val_loss:.4f} Best: {best_val_loss:.4f} Time: {total:.4f}")
            if scheduler is not None:
                scheduler.step(val_loss)

            if epoch % 50 == 49:
                print(f"Total time passed: {((timeit.default_timer() - beg_time) / 60):.4f}")

        torch.save(best_weights, f"models/new_local_fold_{fold}.pth")